In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import neighbors, datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import model_selection

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
churn = pd.read_csv('customer_churn.csv')
churn.head()
churn.info()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
State             3333 non-null object
Account Length    3333 non-null int64
Area Code         3333 non-null int64
Phone             3333 non-null object
Int'l Plan        3333 non-null object
VMail Plan        3333 non-null object
VMail Message     3333 non-null int64
Day Mins          3333 non-null float64
Day Calls         3333 non-null int64
Day Charge        3333 non-null float64
Eve Mins          3333 non-null float64
Eve Calls         3333 non-null int64
Eve Charge        3333 non-null float64
Night Mins        3333 non-null float64
Night Calls       3333 non-null int64
Night Charge      3333 non-null float64
Intl Mins         3333 non-null float64
Intl Calls        3333 non-null int64
Intl Charge       3333 non-null float64
CustServ Calls    3333 non-null int64
Churn?            3333 non-null object
dtypes: float64(8), int64(8), object(5)
memory usage: 546.9+ KB


In [3]:
unique_states = churn['State'].unique()
unique_states.sort()

result = []
counter = 1
for state in unique_states:
    result.append((state, counter))
    counter += 1
d_result = dict(result)

churn['State'].replace(d_result, inplace=True)
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,17,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,36,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,32,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,36,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,37,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [4]:
# Drop account_length, area_code, phone
# Play around with using the Mins, Calls or Charge
churn.columns = churn.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('?','').str.replace('\'','')
churn.churn.replace(['True.','False.'], [1, 0], inplace=True)
churn.intl_plan.replace(['yes','no'], [1, 0], inplace=True)
churn.vmail_plan.replace(['yes','no'], [1, 0], inplace=True)
churn.head()

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn
0,17,128,415,382-4657,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,36,107,415,371-7191,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,32,137,415,358-1921,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,36,84,408,375-9999,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,37,75,415,330-6626,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [11]:
# Add engineered attribute total_calls to determine the total number of calls
# Need to determine if this column will include or exclude custserv_calls
churn['total_calls'] = churn['day_calls'] + churn['eve_calls'] + churn['night_calls']
churn['total_charge'] = churn['day_charge'] + churn['eve_charge'] + churn['night_charge'] + churn['intl_charge']
churn.head()

,state,account_length,area_code,phone,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn,total_calls,total_charge
0,17,128,415,382-4657,0,1,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,0,303,75.56
1,36,107,415,371-7191,0,1,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,332,59.24
2,32,137,415,358-1921,0,0,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,333,62.29
3,36,84,408,375-9999,1,0,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,0,255,66.80
4,37,75,415,330-6626,1,0,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,0,359,52.09


In [25]:
# Create new churn df without redundant columns
new_churn = churn[['state', "intl_plan", 'vmail_plan',
                   'vmail_message', 'total_calls', 'intl_calls', 'total_charge',
                   'custserv_calls', 'churn']].copy()
new_churn.head()

,state,intl_plan,vmail_plan,vmail_message,total_calls,intl_calls,total_charge,custserv_calls,churn
0,17,0,1,25,303,3,75.56,1,0
1,36,0,1,26,332,3,59.24,1,0
2,32,0,0,0,333,5,62.29,0,0
3,36,1,0,0,255,7,66.80,2,0
4,37,1,0,0,359,3,52.09,3,0


In [26]:
X = new_churn.drop('churn',axis=1)
y = new_churn['churn']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
predictions = dtree.predict(X_test)

In [33]:
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))
accuracy_score(y_test, predictions)

             precision    recall  f1-score   support

          0       0.96      0.96      0.96       852
          1       0.78      0.77      0.77       148

avg / total       0.93      0.93      0.93      1000

[[819  33]
 [ 34 114]]


0.933

In [35]:
X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size = 0.30)
model = GaussianNB()
model.fit(X_Train, y_Train)
y_pred = model.predict(X_Test)
print(classification_report(y_Test, y_pred))
accuracy_score(y_test, y_pred)

GaussianNB(priors=None)

             precision    recall  f1-score   support

          0       0.91      0.93      0.92       869
          1       0.45      0.40      0.43       131

avg / total       0.85      0.86      0.85      1000



0.776